Create a model using built-in library from Pytorch.
This code closelt follows nn_tutorial notebook.

In [1]:
import csv, torch, math

### read in target and attr input files
train_target_file = 'train_target_in.csv'
train_attr_file = 'train_attr_in.csv'
valid_target_file = 'valid_target_in.csv'
valid_attr_file = 'valid_attr_in.csv'

def read_data(target_file, attr_file):
    
    with open(target_file, 'r') as f: 
        reader = csv.reader(f)
        target_input_list = list(reader)

    with open(attr_file, 'r') as f: 
        reader = csv.reader(f)
        attr_input_list = list(reader)
        
    target_input = [''.join(sublist) for sublist in target_input_list]
    attr_input = [[float(x) for x in t] for t in attr_input_list]
    
    coded_y = []
    for i in target_input:
        if i == 's': 
            coded_y.append(0)
        elif i == 'x':
            coded_y.append(1)
        else:
            coded_y.append(2)


    ## convert arrays and lists into tensors for training
    return map(torch.tensor, (attr_input, coded_y))

x_train, y_train = read_data(train_target_file, train_attr_file)
x_valid, y_valid = read_data(valid_target_file, valid_attr_file)


x_train and y_train should be in the right shapes.

In [2]:
print(x_train.shape, y_train.shape)
print(x_valid.shape, y_valid.shape)

torch.Size([1568, 3827]) torch.Size([1568])
torch.Size([784, 3827]) torch.Size([784])


target_input_list is in list of lists. We want it to be a list of characters.
attr_input is also a list of string lists. We want it to be a list of float lists.

In [3]:
def accuracy(out, yb):
    preds = torch.argmax(out, dim = 1)
    #print('---', preds, yb)
    return (preds == yb).float().mean()


Since there are three classes, we set D_out to 3. n is total number of instances and c is the number of attributes in each instance. We use a loss function from torch.nn.functional.

In [4]:
train_n, c = x_train.shape
valid_n, c = x_valid.shape
D_out = 3
lr = 0.5
epochs = 10
train_bs = 5
validate_bs = train_bs*2

import torch.nn.functional as F

loss_func = F.cross_entropy


In [6]:
from torch import nn
from torch import optim

class SoundRecognition(nn.Module):
    def __init__(self):
        super().__init__()
        
        ## This one line does the same thing as the two lines below 
        ## and [xb @ self.weights + self.bias]
        self.lin = nn.Linear(c, D_out)
#         self.weights = nn.Parameter(torch.randn(c, D_out) / math.sqrt(c))
#         self.bias = nn.Parameter(torch.zeros(D_out))
        
    def forward(self, xb):
        return self.lin(xb)
    
class SoundRecognition_CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=2)
        self.conv2 = nn.Conv2d(16, 16, kernel_size=3, stride=2)
        self.conv3 = nn.Conv2d(16, 3, kernel_size=3, stride=2)

    def forward(self, xb):
        xb = xb.view(-1, 1, 28, 28)
        xb = F.relu(self.conv1(xb))
        xb = F.relu(self.conv2(xb))
        xb = F.relu(self.conv3(xb))
        xb = F.avg_pool2d(xb, 4)
        return xb.view(-1, xb.size(1))

## Get the model and optim object that will be used to update model parameters
def get_model():
    model = SoundRecognition()
    return model, optim.SGD(model.parameters(), lr = lr)

In [12]:
model, opt = get_model()

def fit():
    for epoch in range(epochs):
        
        print('Training')
        model.train()
        for i in range((train_n - 1) // train_bs + 1):
            
            start_i = i * train_bs
            end_i = start_i + train_bs
            xb = x_train[start_i:end_i]
            yb = y_train[start_i:end_i]
            pred = model(xb)
            loss = loss_func(pred, yb)

            loss.backward()
            opt.step()
            opt.zero_grad()
        
            #print('Acc: ', accuracy(pred, yb))
        
        ####validate at each epoch
        print('Validating')
        model.eval()
        loss = []
        acc = []
        with torch.no_grad():
#             for i in range((valid_n - 1) // validate_bs + 1):
#                 start_i = i * validate_bs
#                 end_i = start_i + validate_bs
#                 xb = x_valid[start_i:end_i]
#                 yb = y_valid[start_i:end_i]
#                 pred = model(xb)
#                 #print('----', yb)
#                 loss.append(loss_func(pred, yb))
#                 acc.append(accuracy(pred, yb))

            for i in range((train_n - 1) // train_bs + 1):

                start_i = i * train_bs
                end_i = start_i + train_bs
                xb = x_train[start_i:end_i]
                yb = y_train[start_i:end_i]
                pred = model(xb)
                loss.append(loss_func(pred, yb))
                acc.append(accuracy(pred, yb))
        
        #print(loss)
        valid_loss = sum(loss)
        valid_acc = sum(acc)
        
    
        print('Epoch: ', epoch, ' | Loss: ', valid_loss, ' | Accuracy: ', valid_acc/len(y_train))
    

fit()


Training
Validating
Epoch:  0  | Loss:  tensor(79198592.)  | Accuracy:  tensor(0.1369)
Training
Validating
Epoch:  1  | Loss:  tensor(86558168.)  | Accuracy:  tensor(0.1534)
Training
Validating
Epoch:  2  | Loss:  tensor(85399288.)  | Accuracy:  tensor(0.1501)
Training
Validating
Epoch:  3  | Loss:  tensor(64007316.)  | Accuracy:  tensor(0.1526)
Training
Validating
Epoch:  4  | Loss:  tensor(81025896.)  | Accuracy:  tensor(0.1403)
Training
Validating
Epoch:  5  | Loss:  tensor(44156648.)  | Accuracy:  tensor(0.1569)
Training
Validating
Epoch:  6  | Loss:  tensor(64956764.)  | Accuracy:  tensor(0.1564)
Training
Validating
Epoch:  7  | Loss:  tensor(60913756.)  | Accuracy:  tensor(0.1443)
Training
Validating
Epoch:  8  | Loss:  tensor(59048036.)  | Accuracy:  tensor(0.1505)
Training
Validating
Epoch:  9  | Loss:  tensor(59307216.)  | Accuracy:  tensor(0.1559)
